# Use of Python in Groundwater Hydrology
# Session 2 - Data processing and plotting

The objective of this notebook is to demonstrate the incredible power of the Pandas library for processing time series data. We will also see how to create graphs in Matplotlib. For this exercise we will make use of online data that can be directly accessed from within Python.

First we import some libraries, including the Python core library `requests`, which can be used to get online files and data.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

Let's get some climate data available from the database of the Royal Dutch Meteorological Institute (KNMI). The data are made available through a so-called application programming interface (API). Details can be found <A href="https://www.knmi.nl/kennis-en-datacentrum/achtergrond/data-ophalen-vanuit-een-script">here</A>, but unfortunately the information is only in Dutch.

In [ ]:
start_date = pd.to_datetime('19010101') 
current_date = pd.to_datetime('today') # Today

url = 'https://www.daggegevens.knmi.nl/klimatologie/daggegevens'

params = {
    'start': start_date.strftime('%Y%m%d'),
    'end': current_date.strftime('%Y%m%d'),
    'inseason': str(int(False)),
    'vars': ":".join(['TG', 'TX', 'RH']),
    'stns': '260', # De Bilt
    'fmt': 'json',
}

# Send the API request
response = requests.get(url, params=params)

The requested file format was a json file, which can be converted to a Pandas DataFrame.

In [ ]:
df = pd.json_normalize(response.json())

We do some manipulation of the DataFrame: We use the 'date' column for the index, we drop the station code (we only obtained data from a single station, so there is no real need to keep it) and we divide the temperatures by ten, because they are reported in tenths of degrees.

In [ ]:

df.index = pd.to_datetime(df['date'])
df = df.drop(['station_code'], axis=1)

df['TG'] = df['TG'] / 10. # Mean daily temperature
df['TX'] = df['TX'] / 10. # Daily maximum temperature.

The daily precipitation sums are stored in the column labelled 'RH'. The next code cell calculates the maximum number of consecutive days in a year without any significant precipitation.

In [ ]:
# Look for -1 values, these represent days with <0.5 mm of precipitation and set to zero.
idx = df['RH'] == -1
df.loc[idx, 'RH'] = 0

# Select only the rows with precipitation larger than zero
idx = df['RH'] > 0
dfr = df.loc[idx, 'RH'].copy()

# Calculate the number of days between the dates in dfr
dfd = dfr.index.to_series().diff().dt.days

# Calculate the maximum number of days per year
dfy = dfd.resample('1Y').max()

# Set the index to the year only
dfy.index = dfy.index.year
dfy.plot.bar(figsize=(12, 8))

The next code cell calculates the deviation of the mean annual temperature from the 1961 - 1990 average.

In [ ]:
# Select only the values for the years 1961 - 1990
idx = (df.index.year >= 1961) & (df.index.year <= 1990)
# Store the daily temperatures in a separate Series and calculate the mean
dfsub = df.loc[idx, 'TG']
tmean = dfsub.mean()
print(tmean)

# Get the yearly averages and subtract the mean
dfy = df['TG'].resample('1Y').mean()
dT = dfy - tmean

# Plot the deviation from the mean
fig, ax = plt.subplots()
ax.plot(dfy.index, dT);

There appears to be a warming trend since the 1960. Let's also plot the atmospheric CO$_2$ levels measured at the Mauna Loa station. The data are available online as a csv file, which can be read directly by Pandas without the need to download it first. Some tweaking of the `read_csv` arguments is needed to import the data in the right way.

In [ ]:
dfco2 = pd.read_csv("https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_mlo.txt", 
    skiprows=58,
    # delimiter=' ',
    delim_whitespace=True,
    header=None,
    names=['year', 'month', 'dec_date', 'monthly_average', 'de-seasonalized', 'days', 'std_days', 'unc_mon_mean'],
    parse_dates=[['year','month']],  
    index_col="year_month",
)

# Plot the data alongside the De Bilt temperature trend
fig, ax = plt.subplots()
ax.plot(dfy.index, dT)
axco2 = ax.twinx()
axco2.plot(dfco2.index, dfco2['monthly_average'], 'C2')

The temperature data can also be plotted as 'warming stripes', which you may have seen before. See <A href="https://www.climate-lab-book.ac.uk/2018/warming-stripes/">https://www.climate-lab-book.ac.uk/2018/warming-stripes/</A>.

In [ ]:
# Scale the vaues
dTmin = dT.min()
dTmax = dT.max()
dTscaled = (dT - dTmin) / (dTmax - dTmin)

# Get the colors from the bwr colormap
from matplotlib import cm
colors =  [cm.bwr(dTs) for dTs in dTscaled]

# Create a bar plot, with each bar being of equal height but with a different colour
fig, ax = plt.subplots(figsize=(8, 1))
ax.bar(dfy.index, np.ones(len(dT)), width=365, color=colors)
ax.axis('off')


Finally let's create a plot of the number of days since the first of January on which a certain temperature was reached. We'll create three graphs, each for a specific temperature (10, 20 and 30 degrees). A `for` loop repeats the code that calculates the number of days.

In [ ]:
# Create a figure with three graphs
fig, axs = plt.subplots(nrows=3, figsize=(8, 9))

# Repeat for temperatures 10, 20 and 30 degrees.
for i, T in enumerate([10, 20, 30]):
    # Select the days with a temperature higher than T
    idx = df['TX'] > T
    dfm = df.loc[idx, 'TX'].to_frame()
    # Calculate the number of days since the first of January
    dfm['doy'] = dfm.index.dayofyear
    # Find the minimum of doy for each year
    dfy = dfm['doy'].resample('1Y').min()

    # Plot a bar graph
    ax = axs[i]
    ax.bar(dfy.index, dfy, width=365)
    ax.set_ylabel(f"Days to reach {T} \u2103")

# Set some properties of the graph to improve their appearance
for ax in axs:
    ax.grid(ls=':')
    if ax != axs[-1]:
        ax.set_xticklabels([])
    else:
        ax.set_xlabel('Year')

# Let Matplotlib optimize the use of space
plt.tight_layout()

### ***Bonus material***: Creating a kmz file with hydrographs

This final code cell is a demonstration of how data from an Excel file can be read, plotted and saved to separate graphs (png format). It is even possible to store link these figures to the well coordinates in a kmz file, which can be opened in Google Earth. For this you'll need the simplekml library, which can be installed with
 
 pip install simplekml

 or (when using conda)

 conda install -c conda-forge simplekml
 

In [ ]:
import simplekml
from pathlib import Path
kmlfile = simplekml.Kml()

# Create hydrographs subfolder (for output) if it does not yet exist
Path('hydrographs/').mkdir(exist_ok=True)

xldata = pd.read_excel(
    'ameland_exercise.xlsx', 
    sheet_name=None, 
    index_col=0, 
    parse_dates=True,
)
xydata = pd.read_excel(
    'ameland_exercise.xlsx', 
    sheet_name="coordinates",
)

def plot_df(df):
    fig, ax = plt.subplots()
    ax.plot(df.index, df["head (cm amsl)"])
    ax.set_ylabel('h (cm)')
    ax.set_title(sheet)
    fname = f"hydrographs/{sheet}.png"
    plt.savefig(fname)
    
    return fname

for i, sheet in enumerate(xldata):
    if (i == 0):
        continue
    
    well_id = sheet[:-3]
    idx = xydata['well_id'] == well_id
    x = xydata['long'].loc[idx].values[0]
    y = xydata['lat'].loc[idx].values[0]

    fname = plot_df(xldata[sheet])

    path = kmlfile.addfile(fname)
    point = kmlfile.newpoint(name=well_id, coords=[[str(x), str(y)]])
    point.description = '<img src="' + path +'" alt={well_id} width="400" height="300" align="left" />'
    
kmlfile.savekmz('hydrographs/ameland.kmz', format=False) 

This notebook was presented by Vincent Post on 22 June 2023 as part of the two-session webinar series titled "Use of Python in Groundwater Hydrology", organised by the International Association of Hydrogeologists (IAH) Lebanon Chapter in collaboration with the Department of Geology at the American University of Beirut.